In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

### DataFrame数据
无论是创建的数据还是外部数据，首先考虑的是如何将其转换为DataFrame数据:

In [2]:
data = {
    'name':['sandy','leon','faye'],
    'year':['2001','2000','2002'],
    'city':['Beijing','Shanghai','Shenzhen']
}

df = DataFrame(data)        # 指定列名和索引: DataFrame(data, columns=[], index=[]) 
df

,name,year,city
0,sandy,2001,Beijing
1,leon,2000,Shanghai
2,faye,2002,Shenzhen


In [3]:
df.values

array([['sandy', '2001', 'Beijing'],
       ['leon', '2000', 'Shanghai'],
       ['faye', '2002', 'Shenzhen']], dtype=object)

In [4]:
df.index

RangeIndex(start=0, stop=3, step=1)

DataFrame中的一列，可以按字典型标记或属性那样检索为Series:

In [5]:
df['name']        # 等价于df.name

0    sandy
1     leon
2     faye
Name: name, dtype: object

In [6]:
# 新增一列，判断city是否为Shanghai，此时不能用df.isShanghai的语法  
df['isShanghai'] = df.city == 'Shanghai'
df

,name,year,city,isShanghai
0,sandy,2001,Beijing,False
1,leon,2000,Shanghai,True
2,faye,2002,Shenzhen,False


In [7]:
del df['isShanghai']     # 不能用df.isShanghai的语法
df.columns

Index(['name', 'year', 'city'], dtype='object')

从DataFrame中选取的列是数据的视图，而不是拷贝。因此，对Series的修改会映射到DataFrame中。如果需要复制，应显式地使用Series的copy方法！！！

In [8]:
# 对于嵌套字典，pandas会将字典的键作为列，将内部字典的键作为行索引
pop = {'Nevada': {2001: 2.4, 2002: 2.9}, 'Ohio': {2000: 1.7, 2001: 1.5, 2002: 3.6}}
frame = DataFrame(pop)
frame

,Nevada,Ohio
2001,2.4,1.5
2002,2.9,3.6
2000,NaN,1.7


### 重建索引

In [9]:
obj1 = Series([4, 2, 1, 3], index=['d', 'b', 'a', 'c'])
obj2 = obj1.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a    1.0
b    2.0
c    3.0
d    4.0
e    NaN
dtype: float64

对于顺序数据，比如时间序列，在重建索引时可能需要进行插值或填值:

In [10]:
obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])     # 索引不连续
obj3.reindex(range(6), method='ffill')   # 向前填充

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

reindex可以改变行索引、列索引，也可以同时改变两者，当仅传入一个序列时，结果中的行会重建索引:

In [11]:
frame1 = DataFrame(np.arange(9).reshape(3, 3), index=['a', 'c', 'd'], columns=['Beijing', 'Shanghai', 'Shenzhen'])
frame2 = frame1.reindex(['a', 'b', 'c', 'd'])     # 重建行索引
print(frame2)
print('-------------------------------')
frame3 = frame1.reindex(columns=['Shanghai', 'Shenzhen', 'Suzhou'])     # 重建列索引
print(frame3)

   Beijing  Shanghai  Shenzhen
a      0.0       1.0       2.0
b      NaN       NaN       NaN
c      3.0       4.0       5.0
d      6.0       7.0       8.0
-------------------------------
   Shanghai  Shenzhen  Suzhou
a         1         2     NaN
c         4         5     NaN
d         7         8     NaN


### 轴向上删除条目

In [12]:
obj = Series(np.arange(5.0), index=['a', 'b', 'c', 'd', 'e'])
newObj1 = obj.drop('c')
newObj2 = obj.drop(['a', 'e'])      # 要加中括号
print(newObj1.index)
print(newObj2.index)

Index(['a', 'b', 'd', 'e'], dtype='object')
Index(['b', 'c', 'd'], dtype='object')


在DataFrame中，索引值可以从轴向上删除:

In [13]:
data = DataFrame(np.arange(16).reshape(4, 4), index=['sh', 'sz', 'bj', 'gz'], columns=['one', 'two', 'three', 'four'])
data.drop(['bj', 'gz'])    # 删除行

,one,two,three,four
sh,0,1,2,3
sz,4,5,6,7


In [14]:
data.drop('two', axis=1)     # 删除列

,one,three,four
sh,0,2,3
sz,4,6,7
bj,8,10,11
gz,12,14,15


In [15]:
data.drop(['two', 'three'], axis=1, inplace=True)    # inplace属性为True时，会清除被删除的数据
data

,one,four
sh,0,3
sz,4,7
bj,8,11
gz,12,15


### 索引、选择与过滤
普通的Python切片中是不包含尾部的，Series的切片是包含尾部的:

In [16]:
data = DataFrame(np.arange(16).reshape(4, 4), index=['sh', 'sz', 'bj', 'gz'], columns=['one', 'two', 'three', 'four'])
data.two

sh     1
sz     5
bj     9
gz    13
Name: two, dtype: int32

In [17]:
data[['three', 'one']]

,three,one
sh,2,0
sz,6,4
bj,10,8
gz,14,12


In [18]:
data[data['three'] > 5]

,one,two,three,four
sz,4,5,6,7
bj,8,9,10,11
gz,12,13,14,15


In [19]:
data < 5

,one,two,three,four
sh,True,True,True,True
sz,True,False,False,False
bj,False,False,False,False
gz,False,False,False,False


In [20]:
data[data < 5] = 0
data

,one,two,three,four
sh,0,0,0,0
sz,0,5,6,7
bj,8,9,10,11
gz,12,13,14,15


### 使用loc和iloc选择数据(行, 列)
针对DataFrame在行上的标签索引，可以使用轴标签(loc)或整数标签(iloc)以NumPy风格的语法选出数组的行和列的子集:

In [21]:
data.loc['sh', ['two', 'four']]         # 等价于 data.iloc[0, [1, 3]], 选择第1行的第2列和第4列

two     0
four    0
Name: sh, dtype: int32

In [22]:
data.loc[:'sz', 'two']      # 用于切片

sh    0
sz    5
Name: two, dtype: int32

In [23]:
data.iloc[:, :3][data.three > 5]

,one,two,three
sz,0,5,6
bj,8,9,10
gz,12,13,14


### 算术和数据对齐
当将对象相加时，如果存在某个索引对不相同，则返回结果的索引将是索引对的并集:

In [24]:
df1 = DataFrame(np.arange(12.0).reshape(3, 4), columns=list('abcd'))
df2 = DataFrame(np.arange(20.0).reshape(4, 5), columns=list('abcde'))
df2.loc[1, 'b'] = np.nan
df1 + df2          # 直接相加会导致在一些不重叠的位置出现NA值

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [25]:
df1.add(df2, fill_value=0)       # sub 减法; div 除法; floordiv 整除; mul 乘法; pow 幂次方

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


### DataFrame和Series之间的操作
DataFrame和Series之间的算术操作与NumPy中不同维度数组间的操作类似:

In [26]:
arr = np.arange(12.0).reshape(3, 4)
arr - arr[0]               # 当从arr中减去arr[0]时，减法在每一行都进行了操作，这就是所谓的广播机制

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

In [27]:
data = DataFrame(np.arange(16).reshape(4, 4), index=['sh', 'sz', 'bj', 'gz'], columns=['one', 'two', 'three', 'four'])
ser = data.iloc[0]      
ser

one      0
two      1
three    2
four     3
Name: sh, dtype: int32

默认情况下，DataFrame和Series的数组操作中会将Series的索引和DataFrame的列进行匹配，并广播到各行:

In [28]:
data - ser

,one,two,three,four
sh,0,0,0,0
sz,4,4,4,4
bj,8,8,8,8
gz,12,12,12,12


如果一个索引值不在DataFrame的列中，也不在Series的索引中，则对象会重建索引并形成联合：

In [29]:
ser2 = Series(range(3), index=['one', 'two', 'five'])
data + ser2

,five,four,one,three,two
sh,NaN,NaN,0.0,NaN,2.0
sz,NaN,NaN,4.0,NaN,6.0
bj,NaN,NaN,8.0,NaN,10.0
gz,NaN,NaN,12.0,NaN,14.0


如果想在列上进行广播，在行上匹配，则必须使用算数方法中的一种:

In [30]:
ser3 = data['two']
print(ser3)
print('-------------------------')
print(data)
print('-------------------------')
print(data.sub(ser3, axis='index'))

sh     1
sz     5
bj     9
gz    13
Name: two, dtype: int32
-------------------------
    one  two  three  four
sh    0    1      2     3
sz    4    5      6     7
bj    8    9     10    11
gz   12   13     14    15
-------------------------
    one  two  three  four
sh   -1    0      1     2
sz   -1    0      1     2
bj   -1    0      1     2
gz   -1    0      1     2


### 函数应用和映射
NumPy的通用函数(逐元素数组方法)对pandas对象也有效:

In [31]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'), index=['sh', 'sz', 'bj', 'gz'])
np.abs(frame)

,b,d,e
sh,0.598793,0.271145,0.044835
sz,1.011995,0.706623,0.358709
bj,0.909317,1.114217,2.077145
gz,0.740171,0.695624,0.269519


另一种常见的操作是将函数应用到一行或者一列的一维数组上，DataFrame的apply方法可以实现这个功能:

In [32]:
f = lambda x: x.max()-x.min()
frame.apply(f)

b    1.752167
d    0.843071
e    2.032310
dtype: float64

In [33]:
frame.apply(f, axis=1)

sh    0.869938
sz    1.370704
bj    3.191362
gz    1.435795
dtype: float64

传递给apply的函数并不一定要返回一个标量值，也可以返回带有多个值的Series:

In [34]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])

frame.apply(f)

,b,d,e
min,-1.011995,-1.114217,0.044835
max,0.740171,-0.271145,2.077145


逐元素的Python函数也可以使用，假设需将frame中的每个浮点数计算一个格式化字符串，可以使用applymap方法：

In [35]:
format = lambda x : '%.2f' %x
frame.applymap(format)

,b,d,e
sh,0.60,-0.27,0.04
sz,-1.01,-0.71,0.36
bj,-0.91,-1.11,2.08
gz,0.74,-0.70,0.27


In [36]:
frame['e'].apply(format)

sh    0.04
sz    0.36
bj    2.08
gz    0.27
Name: e, dtype: object

### 排序和排名
默认情况下，所有的缺失值都会被排序至Series的尾部。

In [37]:
obj = Series(range(4), index=list('dabc'))
print(obj.sort_index())      # 按索引排序
print('------------')
print(obj.sort_values())     # 按值排序

a    1
b    2
c    3
d    0
dtype: int64
------------
d    0
a    1
b    2
c    3
dtype: int64


In [38]:
frame = DataFrame(np.arange(8).reshape(2, 4), index=['three', 'one'], columns=list('dabc'))
frame.sort_index()           # 传入 ascending=False 可按降序排序

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [39]:
frame.sort_index(axis=1)      # 按列索引排序

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [40]:
frame.sort_values(by='b')

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [41]:
frame.sort_values(by=['a', 'b'])

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


### 描述性统计的概述与计算

In [42]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]], index=list('abcd'), columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [43]:
df.sum()            # 传入 axis='columns' 可按行加总

one    9.25
two   -5.80
dtype: float64

除非在整个切片上都是NA，否则NA是会被自动排除的，可以通过禁用skipna来实现不排除NA值:

In [44]:
df.mean(axis='columns') 

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

In [45]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

In [46]:
df.idxmax()    # idxmin, idxmax: 分别计算最小值和最大值所在的索引标签

one    b
two    d
dtype: object

**描述性统计和汇总统计常用方法：**
- argmin，argmax 分别计算最小值和最大值所在的索引位置（整数）
- quantile 计算样本的从0到1间的分位数
- median 中位数（50%分位数）
- mad 平均值的平均绝对偏差
- prod 所有值的积
- var 值的样本方差
- std 值的样本标准差
- skew 样本偏度
- kurt 样本峰度
- diff 计算第一个算数差值（对时间序列有用）